# 11. CHICAGO AIRBNB: OUTLIERS' EFFECT
---

## 1. The Data

In [5]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 99)
pd.set_option("display.max_rows", 999)
pd.set_option('precision', 3)

chic_cols = ['accommodates', 'bedrooms', 'baths', 'beds', 
             'min_nights', 'max_nights', 'n_reviews', 'price']
chicago = pd.read_csv('data/chicago_airbnb1')
chicago = chicago[chic_cols].copy()
print(chicago.shape)
chicago.head()

(6550, 8)


,accommodates,bedrooms,baths,beds,min_nights,max_nights,n_reviews,price
0,1,1.0,0.5,1.0,2,90,178,66.0
1,7,3.0,1.0,5.0,2,60,395,94.0
2,2,1.0,1.0,1.0,2,60,384,61.0
3,4,1.0,1.0,0.0,4,180,49,67.0
4,2,1.0,0.5,1.0,1,1125,44,22.0


In [6]:
chicago.isnull().sum()

accommodates      0
bedrooms        522
baths             6
beds             37
min_nights        0
max_nights        0
n_reviews         0
price             0
dtype: int64

## 2. Handling Missing Data

In [7]:
chicago.groupby(['accommodates'])['baths'].value_counts(dropna=False).head(12)

accommodates  baths
0             NaN         1
1             0.50      202
              1.00      163
              0.75       30
              2.00       14
              1.50       12
              1.25        4
              11.00       3
              0.00        2
              0.25        2
              2.50        2
2             1.00     1392
Name: baths, dtype: int64

In [8]:
mode = chicago['baths'].mode()[0]
chicago['baths'] = chicago.groupby(['accommodates'])['baths'].\
apply(lambda x:x.fillna(x.value_counts().idxmax() 
                        if x.value_counts().max() >=1
                       else mode , inplace = False
                       )
     )
chicago['baths'].isnull().sum()

0

In [9]:
chicago.groupby(['accommodates', 'baths'])['beds'].value_counts(dropna=False).head(12)

accommodates  baths  beds
0             1.00   NaN       1
1             0.00   1.0       1
                     2.0       1
              0.25   0.0       1
                     1.0       1
              0.50   1.0     186
                     2.0       7
                     0.0       6
                     NaN       1
                     3.0       1
                     4.0       1
              0.75   1.0      27
Name: beds, dtype: int64

In [10]:
mode = chicago['beds'].mode()[0]
chicago['beds'] = chicago.groupby(['accommodates', 'baths'])['beds'].\
apply(lambda x:x.fillna(x.value_counts().idxmax() 
                        if x.value_counts().max() >=1
                       else mode , inplace = False
                       )
     )
chicago['beds'].isnull().sum()

0

In [11]:
mode = chicago['bedrooms'].mode()[0]
chicago['bedrooms'] = chicago.groupby(['accommodates', 'beds', 'baths'])['bedrooms'].\
apply(lambda x:x.fillna(x.value_counts().idxmax() 
                        if x.value_counts().max() >=1
                       else mode , inplace = False
                       )
     )
chicago['bedrooms'].isnull().sum()

0

In [12]:
chicago.isnull().sum()

accommodates    0
bedrooms        0
baths           0
beds            0
min_nights      0
max_nights      0
n_reviews       0
price           0
dtype: int64

## 3. Capping Outliers

In [13]:
chicago.max() - chicago.min()

accommodates       16.0
bedrooms           11.0
baths              12.5
beds               30.0
min_nights        499.0
max_nights       8999.0
n_reviews         627.0
price           10000.0
dtype: float64

While it's tempting to leave our target attribute (`price`) untached, it's important to know that if we do it, it will lead to our model wanting reproduce the outlier effect. So let's first normalize the data and cap outliers and then observe the difference

In [15]:
from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()
chicago_1 = normalizer.fit_transform(chicago)
chicago_1 = pd.DataFrame(chicago_1)
chicago_1.columns = chic_cols
chicago_1.head()

SyntaxError: invalid syntax (<ipython-input-15-b24f89298f3f>, line 1)